# MotionBERT Real-Time Application using Streamlit

In [ ]:
#Install streamlit and import libraries
! pip install streamlit

import streamlit as st
import subprocess
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [ ]:
# Clone repositories
subprocess.run(["git", "clone", "https://github.com/Walter0807/MotionBERT"])
subprocess.run(["git", "clone", "https://github.com/MVIG-SJTU/AlphaPose"])

CompletedProcess(args=['git', 'clone', 'https://github.com/MVIG-SJTU/AlphaPose'], returncode=0)

In [ ]:
! pip install -q pyyaml==6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
import torch
print(torch.__version__)
import yaml, scipy
print(yaml.__version__)
print(scipy.__version__)

2.2.1+cu121
6.0
1.11.4


In [ ]:
#Install additional dependencies
!python -m pip install cython
!sudo apt-get install libyaml-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libyaml-doc
The following NEW packages will be installed:
  libyaml-dev
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-dev amd64 0.2.2-1build2 [62.8 kB]
Fetched 62.8 kB in 1s (55.0 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libyaml-dev:amd64.
(Rea

## Alphapose

In [ ]:
os.chdir('/content/AlphaPose')
print(os.getcwd())

/content/AlphaPose


In [ ]:
#Run Alphapose Setup
subprocess.run(["python", "setup.py", "build develop"])

CompletedProcess(args=['python', 'setup.py', 'build develop'], returncode=1)

In [ ]:
#Download Pretrained models
import requests
import os

# Create directories if they don't exist
os.makedirs('/content/AlphaPose/detector/yolo/data', exist_ok=True)
os.makedirs('/content/AlphaPose/detector/tracker/data', exist_ok=True)
os.makedirs('/content/AlphaPose/pretrained_models', exist_ok=True)

# Define file URLs and destination paths
files = {
    'yolov3-spp.weights': ('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', '/content/AlphaPose/detector/yolo/data/yolov3-spp.weights'),
    'JDE-1088x608-uncertainty': ('1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA', '/content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty'),
    'fast_res50_256x192.pth': ('1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn', '/content/AlphaPose/pretrained_models/fast_res50_256x192.pth'),
    'halpe26_fast_res50_256x192.pth': ('1S-ROA28de-1zvLv-hVfPFJ5tFBYOSITb', '/content/AlphaPose/pretrained_models/halpe26_fast_res50_256x192.pth')
}

# Download files
for filename, (file_id, destination) in files.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    r = requests.get(url, allow_redirects=True)
    with open(destination, 'wb') as f:
        f.write(r.content)

print("Files downloaded successfully!")


Files downloaded successfully!


In [ ]:
#Install additional dependencies
!pip install -q cython_bbox

  Preparing metadata (setup.py) ... done


In [ ]:
#Function to run AlphaPose and generate 2D keypoints
def process_video(video_path):
    # Run the command to generate 2D keypoints
    os.chdir('/content/AlphaPose')
    try:
        subprocess.run([
            "python", "scripts/demo_inference.py",
            "--cfg", "configs/halpe_26/resnet/256x192_res50_lr1e-3_1x.yaml",
            "--checkpoint", "pretrained_models/halpe26_fast_res50_256x192.pth",
            "--indir", "examples/res/vis",
            "--video", video_path,
            "--save_video"
        ])
        # Assuming the output video is saved in the same directory as the input video
        output_video_path = video_path.replace(".mp4", "_output.mp4")
        return output_video_path
    except Exception as e:
        st.error(f"An error occurred: {e}")

## MotionBERT

In [ ]:
#Install adsitional dependencies for MotionBERT
!pip install -q ipdb
!pip install -q smplx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00


In [ ]:
# Function for MotionBERT inference
def motionbert_inference(vid_path, keypoints_path, out_path):
    os.chdir('/content/MotionBERT')
    subprocess.run([
        'python', 'infer_wild.py',
        '--vid_path', vid_path,
        '--json_path', keypoints_path,
        '--out_path', out_path
    ])

## Built Streamlit app

In [ ]:
# Main function to run the Streamlit app
def main():
    st.title("Pose Estimation App")

    # Upload video file
    uploaded_file = st.file_uploader("Upload a video", type=["mp4", "avi"])

    if uploaded_file is not None:
        # Save the uploaded file
        video_path = os.path.join("uploads", uploaded_file.name)
        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Process the video and generate 2D keypoints
        # keypoints_path = process_video(video_path)
        keypoints_path = '/content/AlphaPose/examples/res/alphapose-results.json'

        if keypoints_path:
            # Perform MotionBERT inference
            out_path = '/content/3dPose_out'
            motionbert_inference(video_path, keypoints_path, out_path)

            # Display the generated 3D pose video
            output_video_path = os.path.join(out_path, os.path.basename(video_path).replace(".mp4", "_3d_output.mp4"))
            if os.path.exists(output_video_path):
                st.video(output_video_path)
            else:
                st.error("Failed to generate 3D pose video.")
        else:
            st.error("Failed to process video and generate keypoints.")

if __name__ == "__main__":
    main()

2024-04-22 21:18:02.371 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
#Install ngrok library to see the streamlit app in browser
!pip install streamlit pyngrok

In [ ]:
!ngrok authtoken 2fTP6rQbRUrnBQDUwlCa54p7DxA_6yP6JEaQ3irwXWZMNHrFp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install -U streamlit pyngrok

# Import ngrok
from pyngrok import ngrok

!ngrok authtoken 2fTP6rQbRUrnBQDUwlCa54p7DxA_6yP6JEaQ3irwXWZMNHrFp

# Clear any existing ngrok tunnels
ngrok.kill()

# Start Streamlit server on port 8501
!streamlit run --server.port 8501 app.py &>/dev/null&

# Get the URL for accessing the Streamlit app
ngrok_tunnel = ngrok.connect(port='8501')
print('Streamlit URL:', ngrok_tunnel.public_url)

# Keep the notebook running to keep the Streamlit app alive
input("Press Enter to terminate...")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
!pip install streamlit

# Start Streamlit server
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.199.206.159:8501

  Stopping...
^C
